# Trimet Weather

## Goal of this sheet

The hope was that our group was going to be able to see how weather affected ridership on Trimet. We're a little short on data, so this is more of a proof of concept. What we have is a daily weather summary and a list  Average weekly ridership numbers, by month, from 2008 through 2017. What I'm attempting to do in this sheet are these steps:

1. Get a monthly number for max temp, min temp, precip, and snowfall. This number isn't super useful, but it will allow us to see monthly and year-on-year trends. More importantly, if we can get the rest of this to work, it will be trivial to use this data if we could get daily ridership numbers.

2. Use this number to see how weather correlates with the monthly numbers we have. We're going to use the max weekly ridership number. We'll use this number * 4, since we have a weekly average, not an actual weekly number. Again, if we could get a real weekly number, we could hone our results.


## Weather data

In [110]:
# Import all libraries needed this exercise
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Enable inline plotting
%matplotlib inline 

In [111]:
# create DataFrame from weather data

pdx_weather = pd.read_csv('Portland_dailyclimatedata.csv', skiprows = 6)

In [112]:
pdx_weather.head()

,YR,MO,Unnamed: 2,1,2,3,4,5,6,7,...,23,24,25,26,27,28,29,30,31,AVG or Total
0,1940,10,TX,M,M,M,M,M,M,M,...,58,57,57,57,56,53,59,59,52,M
1,1940,10,TN,M,M,M,M,M,M,M,...,53,48,41,38,37,45,48,50,46,M
2,1940,10,PR,M,M,M,M,M,M,M,...,0.63,1.03,0,0,T,0.18,0.58,0.50,0.25,M
3,1940,10,SN,M,M,M,M,M,M,M,...,0,0,0,0,0,0,0,0,0,0.0
4,1940,11,TX,52,53,47,55,51,58,56,...,42,44,51,44,45,59,57,45,-,49.1


In [113]:
pdx_weather.columns

Index(['YR', 'MO', 'Unnamed: 2', '1', '2', '3', '4', '5', '6', '7', '8', '9',
       '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21',
       '22', '23', '24', '25', '26', '27', '28', '29', '30', '31',
       'AVG or Total'],
      dtype='object')

In [114]:
pdx_weather.rename(columns={'Unnamed: 2': 'weather_category'}, inplace=True)
pdx_weather.head()

,YR,MO,weather_category,1,2,3,4,5,6,7,...,23,24,25,26,27,28,29,30,31,AVG or Total
0,1940,10,TX,M,M,M,M,M,M,M,...,58,57,57,57,56,53,59,59,52,M
1,1940,10,TN,M,M,M,M,M,M,M,...,53,48,41,38,37,45,48,50,46,M
2,1940,10,PR,M,M,M,M,M,M,M,...,0.63,1.03,0,0,T,0.18,0.58,0.50,0.25,M
3,1940,10,SN,M,M,M,M,M,M,M,...,0,0,0,0,0,0,0,0,0,0.0
4,1940,11,TX,52,53,47,55,51,58,56,...,42,44,51,44,45,59,57,45,-,49.1


I fixed the column that came in as blank to indicate that it is the weather category being measured. Thankfully, they alread have a monthly average/total numbers listed per month, which is exactly what we need since we only have monthly ridership numbers. Since we only have ridership data from July 08 to March 18, I'm going to trim this down. I'm going used 2009-2017, because I'm trying to avoid wasting time getting the selection straight. could just use the row indexes, but I think the dates are clearer, even if it means not using some of our data on this first pass.

This gives us a dataframe with 432 rows, ordered by year and month. Having the data listed on four lines for each month is kind of a pain, but I'm not sure the best way to split that up yet, so I'll leave it for the moment.

In [126]:
pdx_weather_smaller = pdx_weather[(pdx_weather.YR > 2008) & (pdx_weather.YR < 2018)]

pdx_weather_smaller.head()

,YR,MO,weather_category,1,2,3,4,5,6,7,...,23,24,25,26,27,28,29,30,31,AVG or Total
3276,2009,1,TX,52,43,41,40,51,55,56,...,47,39,40,41,43,48,38,39,44,46.2
3277,2009,1,TN,41,33,31,25,35,46,52,...,30,33,33,26,29,33,32,31,33,33.9
3278,2009,1,PR,2.49,0.76,T,0.12,0.11,0.10,0.2,...,0,T,0.07,0,0.06,0.01,0,T,0.01,4.50
3279,2009,1,SN,0,0.4,0,2.3,0,0,0,...,0,T,0.7,0,1.2,0,0,0,0,4.6
3280,2009,2,TX,42,53,58,59,49,50,53,...,49,55,55,46,50,53,-,-,-,50.3


OK, so we now need to get ridership numbers. I'm just going to pull in the month, bus, and max numbers.

In [ ]:
ridership = pd.read_excel('monthly-performance.xlsx', usecols=2, skiprows=9)
ridership.columns = ['date', 'bus_riders', 'max_riders']
ridership.head()

OK, now I'm adding the column with the montly max totals. We could just use the weekly numbers, but reporting our stuff as weekly ridership by month is confusing is just as confusing as when Trimet does it.

In [ ]:
ridership['Monthlthly_Max_Rides'] = ridership.max_riders * 4
ridership.head(5)

In [149]:
pdx_weather_smaller.head()

,YR,MO,weather_category,1,2,3,4,5,6,7,...,23,24,25,26,27,28,29,30,31,AVG or Total
3276,2009,1,TX,52,43,41,40,51,55,56,...,47,39,40,41,43,48,38,39,44,46.2
3277,2009,1,TN,41,33,31,25,35,46,52,...,30,33,33,26,29,33,32,31,33,33.9
3278,2009,1,PR,2.49,0.76,T,0.12,0.11,0.10,0.2,...,0,T,0.07,0,0.06,0.01,0,T,0.01,4.50
3279,2009,1,SN,0,0.4,0,2.3,0,0,0,...,0,T,0.7,0,1.2,0,0,0,0,4.6
3280,2009,2,TX,42,53,58,59,49,50,53,...,49,55,55,46,50,53,-,-,-,50.3


OK, now I need to get the averages and totals from the seperate weather dataframe lines tied to their month. It is a bit ugly, but I'm going to take advantage of the fact that both dfs are in order. 



In [155]:
# making a df that I can mess with

scratch = ridership.copy()

In [156]:
scratch.head()

,date,bus_riders,max_riders,Monthlthly_Max_Rides
0,2009-01-01,1296700.0,643000.0,2572000.0
1,2009-02-01,1301300.0,634400.0,2537600.0
2,2009-03-01,1235300.0,653900.0,2615600.0
3,2009-04-01,1299000.0,679500.0,2718000.0
4,2009-05-01,1292900.0,700900.0,2803600.0


I'm trying a first pass to tie these 2 together by creating common index. The method below does not work. I need to get the weather file to have all the data on one line for this to happen.

In [157]:
scratch['row'] = range(122)
weather_scratch = pdx_weather_smaller.copy()
# weather_scratch.info()
weather_scratch['row'] = range(432)



In [158]:
scratch.merge(weather_scratch, how = 'inner', on = 'row')


,date,bus_riders,max_riders,Monthlthly_Max_Rides,row,YR,MO,weather_category,1,2,...,23,24,25,26,27,28,29,30,31,AVG or Total
0,2009-01-01,1296700.0,643000.0,2572000.0,0,2009,1,TX,52,43,...,47,39,40,41,43,48,38,39,44,46.2
1,2009-02-01,1301300.0,634400.0,2537600.0,1,2009,1,TN,41,33,...,30,33,33,26,29,33,32,31,33,33.9
2,2009-03-01,1235300.0,653900.0,2615600.0,2,2009,1,PR,2.49,0.76,...,0,T,0.07,0,0.06,0.01,0,T,0.01,4.50
3,2009-04-01,1299000.0,679500.0,2718000.0,3,2009,1,SN,0,0.4,...,0,T,0.7,0,1.2,0,0,0,0,4.6
4,2009-05-01,1292900.0,700900.0,2803600.0,4,2009,2,TX,42,53,...,49,55,55,46,50,53,-,-,-,50.3
5,2009-06-01,1222800.0,721000.0,2884000.0,5,2009,2,TN,31,29,...,35,45,35,33,28,32,-,-,-,32.4
6,2009-07-01,1167400.0,729600.0,2918400.0,6,2009,2,PR,0,0,...,0.32,0.34,0.10,0.10,0,T,-,-,-,1.36
7,2009-08-01,1158000.0,691400.0,2765600.0,7,2009,2,SN,0,0,...,0,0,T,0.3,0,0,-,-,-,0.3
8,2009-09-01,1198500.0,744185.0,2976740.0,8,2009,3,TX,50,59,...,48,57,52,60,54,53,54,51,51,52.5
9,2009-10-01,1224600.0,763900.0,3055600.0,9,2009,3,TN,41,41,...,38,42,39,35,39,40,36,34,42,38.1


# Trying to figure out the weather column issue, lots of random stuff below.


In [129]:
pdxw_copy = pdx_weather_smaller.copy()
pdxw_copy['row_index'] = 0
pdxw_copy['max_temp'] = 0
pdxw_copy['min_temp'] = 0
pdxw_copy['precip'] = 0
pdxw_copy['snow'] = 0
pdxw_copy.head(1)

,YR,MO,weather_category,1,2,3,4,5,6,7,...,28,29,30,31,AVG or Total,row_index,max_temp,min_temp,precip,snow
3276,2009,1,TX,52,43,41,40,51,55,56,...,48,38,39,44,46.2,0,0,0,0,0


In [139]:
pdxw_copy.iloc[1:2]

,YR,MO,weather_category,1,2,3,4,5,6,7,...,28,29,30,31,AVG or Total,row_index,max_temp,min_temp,precip,snow
3277,2009,1,TN,41,33,31,25,35,46,52,...,33,32,31,33,33.9,0,0,0,0,0


In [148]:

# for row in pdxw_copy:
#     if (pdxw_copy[row:'weather_category']) == 'TX':
#         pdxw_copy.loc[row:'max_temp'] = pdx.iloc[row:34]
#         if pdxw_copy[row:'weather_category'] == 'TN':
#             pdxw_copy[row:'min_temp'] = pdx.iloc[row:34]
#             if pdxw_copy[row:'weather_category'] == 'PR':
#                 pdxw_copy[row:'precip'] = pdx.iloc[row:34]
#                 if pdxw_copy[row:'weather_category'] == 'SN':
#                     pdxw_copy[row:'snow'] = pdx.iloc[row:34]

for index, row in pdxw_copy.iterrows():
    if row['weather_category'] == 'TX':
        row['max_temp'] = row['AVG or Total']
    elif row['weather_category'] == 'TN':
        row['min_temp'] = row['AVG or Total']
    elif row['weather_category'] == 'PR':
        row['precip'] = row['AVG or Total']
    elif row['weather_category'] == 'SN':
        row['snow'] = row['AVG or Total']
# print(pdxw_copy.iloc[0,2]=='TX')
pdxw_copy.head(10)

,YR,MO,weather_category,1,2,3,4,5,6,7,...,28,29,30,31,AVG or Total,row_index,max_temp,min_temp,precip,snow
3276,2009,1,TX,52,43,41,40,51,55,56,...,48,38,39,44,46.2,0,0,0,0,0
3277,2009,1,TN,41,33,31,25,35,46,52,...,33,32,31,33,33.9,0,0,0,0,0
3278,2009,1,PR,2.49,0.76,T,0.12,0.11,0.10,0.2,...,0.01,0,T,0.01,4.50,0,0,0,0,0
3279,2009,1,SN,0,0.4,0,2.3,0,0,0,...,0,0,0,0,4.6,0,0,0,0,0
3280,2009,2,TX,42,53,58,59,49,50,53,...,53,-,-,-,50.3,0,0,0,0,0
3281,2009,2,TN,31,29,27,30,36,34,34,...,32,-,-,-,32.4,0,0,0,0,0
3282,2009,2,PR,0,0,0,0,T,0.09,0,...,T,-,-,-,1.36,0,0,0,0,0
3283,2009,2,SN,0,0,0,0,0,0,0,...,0,-,-,-,0.3,0,0,0,0,0
3284,2009,3,TX,50,59,54,52,50,51,46,...,53,54,51,51,52.5,0,0,0,0,0
3285,2009,3,TN,41,41,41,33,39,34,34,...,40,36,34,42,38.1,0,0,0,0,0
